<a href="https://colab.research.google.com/github/himasri-24/Diabetic-Vision-Impairment/blob/main/vgg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import zipfile
import matplotlib.pyplot as plt

# Define constants
IMG_SIZE = (128, 128)  # Reduced image size for faster training
BATCH_SIZE = 16  # Reduced batch size to lower memory usage
EPOCHS = 5  # Reduced epochs to speed up training
ZIP_FILE_PATH = "/content/archive.zip"  # Update this path to your ZIP file
EXTRACTED_FOLDER = "/content/extracted_images"  # Folder where images will be extracted

# Check if the ZIP file exists and is valid
if not os.path.exists(ZIP_FILE_PATH):
    raise FileNotFoundError(f"ZIP file '{ZIP_FILE_PATH}' not found. Please upload the correct file.")

try:
    with zipfile.ZipFile(ZIP_FILE_PATH, 'r') as zip_ref:
        zip_ref.extractall(EXTRACTED_FOLDER)
except zipfile.BadZipFile:
    raise zipfile.BadZipFile(f"Error: '{ZIP_FILE_PATH}' is not a valid ZIP file. Please check the file format.")

# Update dataset path
DATASET_PATH = EXTRACTED_FOLDER

# Check if dataset path exists
if not os.path.exists(DATASET_PATH) or not os.path.isdir(DATASET_PATH):
    raise FileNotFoundError(f"Dataset path '{DATASET_PATH}' not found. Please check the extraction process.")

# Set up data generators
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # 20% validation split
)

train_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'  # Use 80% for training
)

val_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'  # Use 20% for validation
)

# Calculate steps per epoch and validation steps
steps_per_epoch = train_generator.samples // BATCH_SIZE
validation_steps = val_generator.samples // BATCH_SIZE

# Load pre-trained VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the pre-trained layers to prevent them from being updated during training
for layer in base_model.layers:
    layer.trainable = False

# Build the model
model = keras.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(128, activation='relu'),  # Fully connected layer
    layers.Dropout(0.5),  # Dropout to reduce overfitting
    layers.Dense(5, activation='softmax')  # 5 classes (adjust based on your dataset)
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Add Early Stopping to prevent unnecessary training
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with calculated steps_per_epoch and validation_steps
history_vgg16 = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,  # Automatically calculated
    validation_steps=validation_steps,  # Automatically calculated
    callbacks=[early_stopping]
)

# Save the model
model.save('diabetic_retinopathy_vgg16.h5')

print("Model training complete and saved as 'diabetic_retinopathy_vgg16.h5'")



Found 2200 images belonging to 5 classes.
Found 550 images belonging to 5 classes.
Epoch 1/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 459s 3s/step - accuracy: 0.5329 - loss: 1.3607 - val_accuracy: 0.6287 - val_loss: 1.0119
Epoch 2/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 0.3750 - loss: 1.3327 - val_accuracy: 0.6305 - val_loss: 1.0166
Epoch 3/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 478s 3s/step - accuracy: 0.5909 - loss: 1.0782 - val_accuracy: 0.6397 - val_loss: 0.9566
Epoch 4/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 0.5625 - loss: 1.2263 - val_accuracy: 0.6415 - val_loss: 0.9555
Epoch 5/5
137/137 ━━━━━━━━━━━━━━━━━━━━ 456s 3s/step - accuracy: 0.6130 - loss: 1.0317 - val_accuracy: 0.6507 - val_loss: 0.9397


Model training complete and saved as 'diabetic_retinopathy_vgg16.h5'
